# **Lab 1: Iterative methods**
**Gustav Grevsten**

# **Abstract**

The purpose of this lab is to implement and test iterative algorithms for finding solutions to both linear and non-linear systems. We implement algorithms for Jacobi Iteration, Gauss-Seidel iteration, and Newtons method for finding zero-solutions for both scalar and vector valued non-linear functions.

# **Set up environment**

In [1]:
# Load neccessary modules.
import numpy as np

# **Introduction**

Iterative methods can be used for solving problems where direct methods are either too costly on time/storage, or for solving non-linear problems for which it may be difficult or impossible to find an exact solution analytically. The key idea is that the iterative process will eventually converge to the sought out solution, thus allowing for an arbitrarily good approximation. For these methods, it is important to make sure that the starting guesses and equations used fulfill the necessary conditions for convergence.

# **Method**

The algorithms we will employ in this lab are Jacobi Iteration, Gauss-Seidel iteration, and Newtons method both for scalar and vector valued functions.

Firstly, Jacobi iteration can be used for finding vector solution $x$ to the equation $Ax = b$, for a given matrix $A$ and vector $b$. The process starts by selecting a starting guess for our initial value $x^0$ (Generally $x^0 = [0,0 ... 0]^T$) and then iterating on the value using the equation

$x^{k+1}_i = \frac{1}{a_{ii}}(b_i - \sum_{j=1}^{n}a_{ij}x_j^k).$

Note that this process is only guaranteed to converge if the matrix $A$ is strictly or irreducibly diagonally dominant (See the course book, chapter 7, page 152). It must therefore hold that

$|a_{ii}| \geq \sum_{i \neq j} |a_{ij}|, \forall i$

and there exists at least one index $k$ such that

$|a_{kk}| > \sum_{i \neq j} |a_{kj}|$

It should be noted that the process may still converge for other matrices.

For more information, see [this article](https://en.wikipedia.org/wiki/Jacobi_method) on Jacobi iteration.

The Jacobi iteration algorithm is implemented as python code below:

In [2]:
def jacobi_iteration(A,b):
  dim = len(b)
  thresh = 10**-6
  x = []
  for i in range(dim):
    x.append(0)
  x = np.array(x)
  while np.linalg.norm(np.matmul(A,x) - b) > thresh:
    x_temp = []
    for i in range(dim):
        sum = 0
        for j in range(dim):
          if i != j:
            sum += A[i,j]*x[j]
        x_temp.append((1/A[i,i])*(b[i] - sum))
    x = np.array(x_temp)
  return x

Next, we will present the Gauss–Seidel method. This method is very similar to Jacobi iteration, except we use the updated values of $x^{k+1}$ as they are consecutively calculated. This means that we can overwrite the vector $x$, and we do not have to store two vectors (one for $x^k$ and one for $x^{k+1}$) as we do in Jacobi iteration, which can be useful for very large problems where much data must otherwise be stored. Each step in the algorithm is defined by the equation

$x^{k+1}_i = \frac{1}{a_{ii}}(b_i - \sum_{j=1}^{i-1}a_{ij}x_j^{k+1} - \sum_{j=i}^{n}a_{ij}x_j^{k}).$

This algorithm follows the same converge criteria of $A$ being strictly or irreducibly diagonally dominant.

For more information, see [this article](https://en.wikipedia.org/wiki/Jacobi_method) on the Gauss–Seidel method.

The Gauss–Seidel method algorithm is implemented as python code below:

In [3]:
def gauss_seidel(A,b):
  dim = len(b)
  thresh = 10**-6
  x = []
  for i in range(dim):
    x.append(0)
  x = np.array(x)
  while np.linalg.norm(np.matmul(A,x) - b) > thresh:
    x_temp = []
    for i in range(dim):
        sum = 0
        for j in range(i):
            sum += A[i,j]*x_temp[j]
        for j in range(i+1,dim):
            sum += A[i,j]*x[j]
        x_temp.append((1/A[i,i])*(b[i] - sum))
    x = np.array(x_temp)
  return x

Next, we present Newton's method for finding roots $x_0$ of real-valued functions such that $f(x_0) = 0$, where the derivative $f'(x)$ can either be directly calculated or approximated. The algorithm is defined by the equation

$x^{k+1} = x^k - \frac{f(x^k)}{f'(x^k)}.$

Newton's method will usually converge for sufficiently close initial guesses $x^0$.

For more information, see [this article](https://en.wikipedia.org/wiki/Newton%27s_method) on Newton's method.

The algorithm for Newtons method is concisely implemented as python code below:

In [4]:
def der(f, x):
  dx = 10**-10
  df = f(x+dx) - f(x)
  return df/dx

def newton(f, guess):
  x = guess
  thresh = 10**-3
  while np.absolute(f(x)) > thresh:
    x -= f(x)/der(f, x)
  return x

Finally, Newtons method can be extended to real vector valued functions $F(\overline{x}) = \overline{y}$. For this method, we use the Jacobian matrix in order to calculate each step, giving us the equation

$\overline{x}^{k+1} = \overline{x}^k - J_F(\overline{x}^k)^{-1}F(\overline{x}^k).$

The algorithm is implemented as python code below:

In [5]:
def vec_der(f, x, index):
  dx = 10**-10
  dv = np.copy(x)
  dv[index] += dx
  df = f(dv) - f(x)
  return df/dx

def jacobian(f,x):
  dim = len(x)
  J = np.zeros((dim, dim))
  for i in range(dim):
    for j in range(dim):
      J[i,j] = vec_der(f, x, j)[i]
  return J

def vec_newton(f, guess):
  x = guess
  thresh = 10**-3
  while np.linalg.norm(f(x)) > thresh:
    Df = jacobian(f,x)
    dx = np.linalg.solve(Df, -f(x))
    x += dx
  return x

# **Results**

We test all of the algorithms presented in the Methods section below. We start by creating an arbitrary vector $b$ and a diagonally dominant matrix $A$:

In [6]:
b = np.array(
    [123,
     456,
     789])

A = np.array([
    [10,2,3],
    [4,15,6],
    [7,8,20]
    ])

x = jacobi_iteration(A,b)

print("Jacobi iteration")
print("x = " + str(x))
print("Ax = " + str(np.matmul(A,x)))
print("b = " + str(b))
print("")

x = gauss_seidel(A,b)

print("Gauss-Seidel")
print("x = " + str(x))
print("Ax = " + str(np.matmul(A,x)))
print("b = " + str(b))

Jacobi iteration
x = [-1.05033707 17.56314608 32.79235957]
Ax = [123.00000021 456.00000038 789.00000054]
b = [123 456 789]

Gauss-Seidel
x = [-1.05033703 17.56314608 32.79235953]
Ax = [123.00000045 456.0000002  789.        ]
b = [123 456 789]


As can be seen, both methods converge to the same values, and give highly accurate results. Next, we test Newton's method applied to the function $cos(x)$

In [7]:
def f(x):
  return np.cos(x)

sol = newton(f, 1)

print("x_0 = " + str(sol))
print("f(x_0) = " + str(f(sol)))

x_0 = 1.5706752856746578
f(x_0) = 0.00012104111994330725


As we can see, the method closely approximated the root $x = \frac{\pi}{2}$. Finally, we test Newton's method for vector valued function $F((x,y,z)^T) = (cos(y),zsin(x)-z,z^2-\frac{\pi}{2})^T$.

In [8]:
def vec_f(x):
  return np.array([np.cos(x[1]), x[2]*np.sin(x[0]) - x[2], x[2]**2 - np.pi**2])

test = vec_newton(vec_f, np.array([1.57, 1, 0.1]))

print("solution x = " + str(test))
print("F(x) = " + str(vec_f(test)))

solution x = [1.58217469 1.57079633 3.14159318]
F(x) = [ 6.12323400e-17 -2.03364333e-04  3.28464474e-06]


Which closely approximates the solution $(x,y,z)^T = (\frac{\pi}{2},\frac{\pi}{2},\pi)^T$

# **Discussion**

As expected, we were able to find good approximate solutions for the various problems using the various iterative methods. Ultimately, this shows that they can be fast and powerful tools for solving hard problems.